### Preprocessing:
    use ./dataset/fastp.sh to clean the sequences first.
### Deduplication:
    Remove duplicate sequences from all N-glycosylation and O-glycosylation sequences.


### Alignment:
Reference Sets:

1. Rfam reference set (including sRNA, snRNA, snoRNA, mature miRNA, tRNA)
2. miRbase miRNA precursor reference set
3. ncRNA from Ensembl

In [ ]:
# Remove duplicate sequences from all N-glycosylation and O-glycosylation sequences.

def read_fasta(file_path):
    sequences = {}
    current_id = None
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                current_id = line[1:]
                sequences[current_id] = ''
            else:
                sequences[current_id] += line
    return sequences

def write_fasta(output_path, sequences):
    with open(output_path, 'w') as file:
        for seq_id, sequence in sequences.items():
            file.write(f'>{sequence}\n{seq_id}\n')

def remove_duplicate_sequences(input_path, output_path):
    sequences = read_fasta(input_path)
    unique_sequences = {sequence: seq_id for seq_id, sequence in sequences.items()}
    write_fasta(output_path, unique_sequences)

# example
# fasta_input_file = 'C1_R1_clean.fa'
# fasta_output_file = 'C1_R1_clean_nonre.fa'
# remove_duplicate_sequences(fasta_input_file, fasta_output_file)

# for i in range(1, 4):
#     fasta_input_file = 'C' + str(i) + '_R1_clean.fa'
#     fasta_output_file = 'C' + str(i) + '_R1_clean_nonre.fa'
#     remove_duplicate_sequences(fasta_input_file, fasta_output_file)
#     fasta_input_file = 'N' + str(i) + '_R1_clean.fa'
#     fasta_output_file = 'N' + str(i) + '_R1_clean_nonre.fa'
#     remove_duplicate_sequences(fasta_input_file, fasta_output_file)

## BLAST
maybe take a lot of time.
#### First, cd to the directory "./dataset/blast" and run the following command to build the blast database:
#### bash set_blastdb.sh

In [ ]:
for i in ['O','N']:
    # for j in ['srna','trna','rrna','snrna','rrna','hairpin']:
    for j in ['pirna']:
        print(i,j)
        !blastn -query ./dataset/{i}/all_nonre.fa -db ./dataset/blastdb/rfam.hg.{j} -outfmt 6 -num_threads 20 -out ./dataset/{i}/{j}-short.txt -perc_identity 100 -task blastn-short -word_size 7 -evalue 1


## Obtain a dataset without duplicates.

In [ ]:
from Bio import SeqIO
def get_reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N','U': 'A','A': 'U'}
    return ''.join([complement[base] for base in seq[::-1]])
def fasta2dict(fasta_path):
    fa={}
    for record in SeqIO.parse(fasta_path, "fasta"):
        fa[record.id]=str(record.seq)
    return fa
N_fa=fasta2dict("../"+'N'+"/"+"all_nonre.fa")
O_fa=fasta2dict("../"+'O'+"/"+"all_nonre.fa")
def get_set(types,t,k):

    blast_file='../'+types+'/'+t+k+".txt"
    # please download the reference file from the supplementary files
    full_fa=fasta2dict("../ref/rfam.hg."+t+".fa")

    if types=="N":
        short_fa=N_fa
    else:
        short_fa=O_fa
    short_set=[]
    long_set=[]
    with open(blast_file, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) >= 2: 
                # print(parts[0], short_fa[parts[0]],parts[1],full_fa[parts[1]])
                short=short_fa[parts[0]];long=full_fa[parts[1]]
                if t=="hairpin":
                    short=short.replace("T","U")
                if short in long:
                    # print("yes")
                    short_set.append(short)
                    long_set.append(long)
                else:
                    if get_reverse_complement(short) in long:
                        short_set.append(get_reverse_complement(short))
                        long_set.append(long)
    print("未经过去重的正例数量：",len(short_set),len(long_set))
    short_set=list(set(short_set))
    long_set=list(set(long_set))
    print("经过去重后的正例数量：短序列：",len(short_set),"全长：",len(long_set))

    import random
    import os
    positive_samples = long_set
    long_set = set(long_set)

    # 使用集合差集操作找出不在long_set中的key对应的value
    all_negative_samples = {value for key, value in full_fa.items() if key not in long_set}

    #  transform short_set to set and judge whether all elements are not in short_set by set intersection
    short_set = set(short_set)
    negative_samples_sel = [i for i in all_negative_samples if not short_set.intersection(i)]


    #  random select the same number of negative samples as positive samples
    if len(negative_samples_sel) < len(positive_samples):
        print("负例数量不足")
        negative_samples = negative_samples_sel
    else:
        
        negative_samples = random.sample(negative_samples_sel, len(positive_samples))
    if os.path.exists("../set/"+types+"/"+t+k+"/"):
        pass
    else:
        os.mkdir("../set/"+types+"/"+t+k+"/")
    with open("../set/"+types+"/"+t+k+'/positive.txt', 'w') as pos_file:
        for sample in positive_samples:
            pos_file.write(sample + '\n')

    with open("../set/"+types+"/"+t+k+'/negative.txt', 'w') as neg_file:
        for sample in negative_samples:
            neg_file.write(sample + '\n')
# for t in ["srna","hairpin","snrna","trna",'rrna']:
for t in ["pirna"]:
    for k in ["-short"]:
        print(t+k)
        print("N")
        get_set("N",t,k)
        print("O")
        get_set("O",t,k)

In [ ]:
# remove duplicate sequences

def convert_to_fasta(file_name, label, output_file):
    with open(file_name, 'r') as file:
        sequences = file.readlines()

    with open(output_file, 'a') as fasta_file:
        for i, sequence in enumerate(sequences):
            sequence = sequence.strip()  # Remove any trailing newline characters
            if 'N' in sequence:
                
                continue
            print( 'N' in sequence)
            header = f">sequence{i+1}|{label}|training\n"
            fasta_file.write(header)
            fasta_file.write(sequence + "\n")
root_path="./dataset/xxxx"
# Convert positive.txt with label 1
convert_to_fasta(root_path+"/positive.txt", 1, root_path+"/output.fasta")

# Convert negative.txt with label 0
convert_to_fasta(root_path+"/negative.txt", 0, root_path+"/output.fasta")

print("Conversion to FASTA format completed.")

!cd-hit-est -i ./dataset/O/all_nonre.fa -o ./dataset/O/CD-all_nonre.fa -M 0 -T 100


In [ ]:
from Bio import SeqIO
# example
# cd ./dataset/O/
with open('./CD-all_nonre.fasta', 'r') as fasta_file:
    sequences = list(SeqIO.parse(fasta_file, 'fasta'))

#  created new positive.txt and negative.txt files
with open('./positive.txt', 'w') as pos_file, open('./positive.txt', 'w') as neg_file:
    for seq in sequences:

        if 'positive' in seq.description:
            pos_file.write(str(seq.seq) + '\n')
        elif 'negative' in seq.description:
            neg_file.write(str(seq.seq) + '\n')

In [ ]:
# now you can use the positive.txt and negative.txt files to train your model
